Check azure-ai-ml

In [1]:
pip show azure-ai-ml

Name: azure-ai-ml
Version: 1.29.0
Summary: Microsoft Azure Machine Learning Client Library for Python
Home-page: https://github.com/Azure/azure-sdk-for-python
Author: Microsoft Corporation
Author-email: azuresdkengsysadmins@microsoft.com
License: MIT License
Location: c:\Users\Tony\AppData\Local\Programs\Python\Python313\Lib\site-packages
Requires: azure-common, azure-core, azure-mgmt-core, azure-monitor-opentelemetry, azure-storage-blob, azure-storage-file-datalake, azure-storage-file-share, colorama, isodate, jsonschema, marshmallow, pydash, pyjwt, pyyaml, strictyaml, tqdm, typing-extensions
Required-by: 
Note: you may need to restart the kernel to use updated packages.


Connect to my workspace

In [2]:
#import required libraries
from azure.ai.ml import MLClient
from azure.identity import DefaultAzureCredential
#Enter    details    of    your    Azure    Machine    Learning    workspace
subscription_id = 'c6cdfd9c-3767-4be7-a343-0acd27ddefb9'
resource_group = 'rg-tshen-0917'
workspace = 'ml-ws-tshen-0917'
#connect to the workspace
ml_client    =    MLClient(DefaultAzureCredential(),    subscription_id,
resource_group,    workspace)
ml_client

MLClient(credential=<azure.identity._credentials.default.DefaultAzureCredential object at 0x0000024326455FD0>,
         subscription_id=c6cdfd9c-3767-4be7-a343-0acd27ddefb9,
         resource_group_name=rg-tshen-0917,
         workspace_name=ml-ws-tshen-0917)

Write local-fix-missing-data.py

In [5]:
%%writefile ./src/local-fix-missing-data.py
# import libraries
import argparse
import glob
from pathlib import Path
import pandas as pd
import mlflow

# get parameters
parser = argparse.ArgumentParser()
parser.add_argument("--input_data", type=str, help='Path to input data')
parser.add_argument('--output_data', type=str, help='Path of output data')
args = parser.parse_args()

# load the data (passed as an input dataset)
# data_path = args.input_data
# all_files = glob.glob(data_path + "/*.csv")
# df = pd.concat((pd.read_csv(f) for f in all_files), sort=False)
df = pd.read_csv(args.input_data)

# log row count input data
row_count = (len(df))
mlflow.log_metric('row count input data', row_count)

# remove nulls
df = df.dropna()

# log processed rows
row_count_processed = (len(df))
mlflow.log_metric('row count output data', row_count_processed)

# set the processed data as output
output_df = df.to_csv(args.output_data)

Overwriting ./src/local-fix-missing-data.py


In [8]:
!python ./src/local-fix-missing-data.py --input_data ./data/diabetes.csv --output_data ./data/diabetes-out.csv

Write local-normalize-data.py

In [10]:
%%writefile ./src/local-normalize-data.py
# import libraries
import argparse
import os
import glob
from pathlib import Path
import pandas as pd
import mlflow
from sklearn.preprocessing import MinMaxScaler

# get parameters
parser = argparse.ArgumentParser()
parser.add_argument("--input_data", type=str, help='Path to input data')
parser.add_argument('--output_data', type=str, help='Path of output data')
args = parser.parse_args()

"""
# load the data (passed as an input dataset)
print("files in input_data path: ")
arr = os.listdir(args.input_data)
print(arr)

for filename in arr:
    print("reading file: %s ..." % filename)
    with open(os.path.join(args.input_data, filename), "r") as handle:
        print(handle.read())

data_path = args.input_data
all_files = glob.glob(data_path + "/*.csv")
df = pd.concat((pd.read_csv(f) for f in all_files), sort=False)
"""

df = pd.read_csv(args.input_data)
# log row count input data
row_count = (len(df))
mlflow.log_metric('row count input data', row_count)

# remove nulls
df = df.dropna()

# log processed rows
row_count_processed = (len(df))
mlflow.log_metric('row count output data', row_count_processed)

# set the processed data as output
output_df = df.to_csv(args.output_data)

# normalize the numeric columns
scaler = MinMaxScaler()
num_cols = ['Pregnancies','PlasmaGlucose','DiastolicBloodPressure','TricepsThickness','SerumInsulin','BMI','DiabetesPedigree']
df[num_cols] = scaler.fit_transform(df[num_cols])

# log processed rows
row_count_processed = (len(df))
mlflow.log_metric('row count output data', row_count_processed)

# set the processed data as output
output_df = df.to_csv(args.output_data)

Overwriting ./src/local-normalize-data.py


Run local-normalize-data.py

In [11]:
!python ./src/local-normalize-data.py --input_data ./data/diabetes.csv --output_data ./data/diabetes-out-2.csv

Write local-train-decision-tree.py

In [24]:
%%writefile ./src/local-train-decision-tree.py
# Import libraries
import argparse
import glob
import pickle
from pathlib import Path

import matplotlib.pyplot as plt
import mlflow
import numpy as np
import pandas as pd
from sklearn.metrics import confusion_matrix, roc_auc_score, roc_curve
from sklearn.model_selection import train_test_split
from sklearn.tree import DecisionTreeClassifier

# get parameters
parser = argparse.ArgumentParser("train")
parser.add_argument("--training_data", type=str, help="Path to training data")
parser.add_argument("--model_output", type=str, help="Path of output model")

args = parser.parse_args()

training_data = args.training_data
model_output = args.model_output

"""
# load the prepared data file in the training folder
print("Loading Data...")
data_path = args.training_data
all_files = glob.glob(data_path + "/*.csv")
df = pd.concat((pd.read_csv(f) for f in all_files), sort=False)
"""

df = pd.read_csv(args.training_data)

# Separate features and labels
X, y = (
    df[
        [
            "Pregnancies",
            "PlasmaGlucose",
            "DiastolicBloodPressure",
            "TricepsThickness",
            "SerumInsulin",
            "BMI",
            "DiabetesPedigree",
            "Age",
        ]
    ].values,
    df["Diabetic"].values,
)

# Split data into training set and test set
X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.30, random_state=0
)

# Train a decision tree model
print('Training a decision tree model...')
model = DecisionTreeClassifier().fit(X_train, y_train)

# Calculate accuracy
y_pred = model.predict(X_test)
acc = np.average(y_pred == y_test)
print("Accuracy:", acc)
mlflow.log_metric("Accuracy", float(acc))

# Calculate AUC
y_pred_proba = model.predict_proba(X_test)
auc = roc_auc_score(y_test, y_pred_proba[:, 1])
print("AUC: " + str(auc))
mlflow.log_metric("AUC", float(auc))

# Plot ROC curve
fpr, tpr, thresholds = roc_curve(y_test, y_pred_proba[:, 1])
fig = plt.figure(figsize=(6, 4))
# Plot the diagonal 50% line
plt.plot([0, 1], [0, 1], "k--")
# Plot the FPR and TPR achieved by our model
plt.plot(fpr, tpr)
plt.xlabel("False Positive Rate")
plt.ylabel("True Positive Rate")
plt.title("ROC Curve")
plt.savefig("ROCcurve.png")
mlflow.log_artifact("ROCcurve.png")

# Create confusion matrix
conf_matrix = confusion_matrix(y_true=y_test, y_pred=y_pred)
fig, ax = plt.subplots(figsize=(7.5, 7.5))
ax.matshow(conf_matrix, cmap=plt.cm.Blues, alpha=0.3)
for i in range(conf_matrix.shape[0]):
    for j in range(conf_matrix.shape[1]):
        ax.text(
            x=j, y=i, s=conf_matrix[i, j], va="center", ha="center",
            size="xx-large"
        )

plt.xlabel("Predictions", fontsize=18)
plt.ylabel("Actuals", fontsize=18)
plt.title("Confusion Matrix", fontsize=18)
plt.savefig("ConfusionMatrix.png")
mlflow.log_artifact("ConfusionMatrix.png")

# Output the model and test data
# Output the model and test data
# pickle.dump(model, open((Path(args.model_output) / "model.sav"), "wb"))
pickle.dump(model, open(model_output, "wb"))




Overwriting ./src/local-train-decision-tree.py


Run local-train-decision-tree.py

In [25]:
!python ./src/local-train-decision-tree.py --training_data ./data/diabetes-out-2.csv --model_output ./model/decision-tree.save

Training a decision tree model...
Accuracy: 0.8846666666666667
AUC: 0.8711328921836907


Write local-train-logistic-regression.py

In [42]:
%%writefile ./src/local-train-logistic-regression.py
# Import libraries
import argparse
import glob
import pickle
from pathlib import Path
import matplotlib.pyplot as plt
import mlflow
import numpy as np
import pandas as pd
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import confusion_matrix, roc_auc_score, roc_curve
from sklearn.model_selection import train_test_split

# get parameters
parser = argparse.ArgumentParser("train")
parser.add_argument("--training_data", type=str, help="Path to training data")
parser.add_argument("--reg_rate", type=float, default=0.01)
parser.add_argument("--model_output", type=str, help="Path of output model")
parser.add_argument("--test_size", type=float, default=0.30, help="test size")
parser.add_argument("--random_state", type=int, default=None, help="random state")

args = parser.parse_args()
print(args)

training_data = args.training_data
model_output = args.model_output
reg_rate = args.reg_rate
test_size = args.test_size
random_state = args.random_state

# load the prepared data file in the training folder
print("Loading Data...")

# data_path = args.training_data
# all_files = glob.glob(data_path + "/*.csv")
# df = pd.concat((pd.read_csv(f) for f in all_files), sort=False)
df = pd.read_csv(training_data)

# Separate features and labels
X, y = (
    df[
        [
            "Pregnancies",
            "PlasmaGlucose",
            "DiastolicBloodPressure",
            "TricepsThickness",
            "SerumInsulin",
            "BMI",
            "DiabetesPedigree",
            "Age",
        ]
    ].values,
    df["Diabetic"].values,
)

# Split data into training set and test set
X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=test_size, random_state=random_state
)

# Train a logistic regression model
print('Training a logistic regression model...')
model = LogisticRegression(C=1 / reg_rate, solver="liblinear").fit(
    X_train, y_train
)

# Calculate accuracy
y_pred = model.predict(X_test)
acc = np.average(y_pred == y_test)
print("Accuracy:", acc)
mlflow.log_metric("Accuracy", float(acc))

# Calculate AUC
y_pred_proba = model.predict_proba(X_test)
auc = roc_auc_score(y_test, y_pred_proba[:, 1])
print("AUC: " + str(auc))
mlflow.log_metric("AUC", float(auc))

# Plot ROC curve
fpr, tpr, thresholds = roc_curve(y_test, y_pred_proba[:, 1])
fig = plt.figure(figsize=(6, 4))
# Plot the diagonal 50% line
plt.plot([0, 1], [0, 1], "k--")
# Plot the FPR and TPR achieved by our model
plt.plot(fpr, tpr)
plt.xlabel("False Positive Rate")
plt.ylabel("True Positive Rate")
plt.title("ROC Curve")
plt.savefig("ROCcurve.png")
mlflow.log_artifact("ROCcurve.png")

# Create confusion matrix
conf_matrix = confusion_matrix(y_true=y_test, y_pred=y_pred)
fig, ax = plt.subplots(figsize=(7.5, 7.5))
ax.matshow(conf_matrix, cmap=plt.cm.Blues, alpha=0.3)
for i in range(conf_matrix.shape[0]):
    for j in range(conf_matrix.shape[1]):
        ax.text(
            x=j, y=i, s=conf_matrix[i, j], va="center", ha="center",
            size="xx-large"
        )

plt.xlabel("Predictions", fontsize=18)
plt.ylabel("Actuals", fontsize=18)
plt.title("Confusion Matrix", fontsize=18)
plt.savefig("ConfusionMatrix.png")
mlflow.log_artifact("ConfusionMatrix.png")

# Output the model and test data
# pickle.dump(model, open((Path(args.model_output) / "model.sav"), "wb"))
pickle.dump(model, open(model_output, "wb"))

Overwriting ./src/local-train-logistic-regression.py


Run local-train-logistic-regression.py

In [41]:
!python ./src/local-train-logistic-regression.py --training_data ./data/diabetes-out-2.csv --model_output ./model/log-regression-tree.save

Namespace(training_data='./data/diabetes-out-2.csv', reg_rate=0.01, model_output='./model/log-regression-tree.save', test_size=0.3, random_state=None)
Loading Data...
Training a logistic regression model...
Accuracy: 0.7813333333333333
AUC: 0.8548116214965853
